In [ ]:
b

In [96]:
import os
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:


data_folder = os.path.join('..', 'data', 'raw')
all_recipe_files = [os.path.join(data_folder, fname)
                    for fname in os.listdir(data_folder)]
all_recipe_files

['../data/raw/sample_submission.csv',
 '../data/raw/test.csv',
 '../data/raw/train.csv']

In [7]:
documents = {}
for text_fname in all_recipe_files:
    bname = os.path.basename(text_fname).split('.')[0]
    print(bname)

sample_submission
test
train


In [9]:

documents = {}
for text_fname in all_recipe_files:
    bname = os.path.basename(text_fname).split('.')[0]
    documents[bname] = pd.read_csv(text_fname, encoding='utf8')


In [10]:
documents

{'sample_submission':           textID  selected_text
 0     f87dea47db            NaN
 1     96d74cb729            NaN
 2     eee518ae67            NaN
 3     01082688c6            NaN
 4     33987a8ee5            NaN
 ...          ...            ...
 3529  e5f0e6ef4b            NaN
 3530  416863ce47            NaN
 3531  6332da480c            NaN
 3532  df1baec676            NaN
 3533  469e15c5a8            NaN
 
 [3534 rows x 2 columns],
 'test':           textID                                               text sentiment
 0     f87dea47db  Last session of the day  http://twitpic.com/67ezh   neutral
 1     96d74cb729   Shanghai is also really exciting (precisely -...  positive
 2     eee518ae67  Recession hit Veronique Branquinho, she has to...  negative
 3     01082688c6                                        happy bday!  positive
 4     33987a8ee5             http://twitpic.com/4w75p - I like it!!  positive
 ...          ...                                                ...     

In [13]:
documents['train']['text'][0]

' I`d have responded, if I were going'

In [20]:
train_corpus_all_in_one = ' '.join([tweet for tweet in documents['train']['text'].apply(lambda x : str(x))])

print(train_corpus_all_in_one[0:100])

 I`d have responded, if I were going  Sooo SAD I will miss you here in San Diego!!! my boss is bully


## Tokenisation

In [21]:


try:  # py3
    all_tokens = [t for t in word_tokenize(train_corpus_all_in_one)]
except UnicodeDecodeError:  # py27
    all_tokens = [t for t in word_tokenize(train_corpus_all_in_one.decode('utf-8'))]

print("Total number of tokens: {}".format(len(all_tokens)))

Total number of tokens: 438371



## Counting Words

We start with a simple word count using `collections.Counter`

We are interested in finding:

  -  how many times a word occurs across the whole corpus (total number of occurrences)
  -  in how many documents a word occurs



In [22]:
from collections import Counter

total_term_frequency = Counter(all_tokens)

for word, freq in total_term_frequency.most_common(20):
    print("{}\t{}".format(word, freq))


!	15296
.	13837
`	11614
I	11467
to	9885
,	8454
the	8431
a	6527
i	5135
my	4975
*	4953
and	4801
you	4794
it	4590
?	4266
...	3875
is	3741
in	3646
for	3597
s	3517


In [30]:
document_frequency = Counter()

for tweet in documents['train']['text'].apply(lambda x : str(x)):
    tokens = word_tokenize(tweet)
    unique_tokens = set(tokens)
    document_frequency.update(unique_tokens)

for word, freq in document_frequency.most_common(20):
    print("{}\t{}".format(word, freq))

.	9380
`	8804
I	8342
!	8129
to	7999
the	7014
,	6496
a	5753
my	4445
and	4328
it	4031
you	3973
i	3816
is	3478
in	3361
for	3336
...	3276
s	3182
?	2963
of	2870


## Stop-words

In [32]:
print(stopwords.words('english'))
print(len(stopwords.words('english')))
print(string.punctuation)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [79]:
stop_list = stopwords.words('english') + list(string.punctuation)

tokens_no_stop = [token for token in all_tokens
                        if token not in stop_list]

total_term_frequency_no_stop = Counter(tokens_no_stop)

for word, freq in total_term_frequency_no_stop.most_common(20):
    print("{}\t{}".format(word, freq))

I	11467
...	3875
..	1670
day	1613
get	1383
like	1302
good	1264
http	1243
go	1209
got	1095
work	1090
today	1054
going	948
love	945
one	918
know	917
time	884
u	847
really	822
back	814


In [72]:
len(tokens_no_stop)

294710

In [47]:
def transform_target(st):
    st =  -1 if st=="negative" else 1 if st=="positive" else 0
    return st

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
import numpy as np

In [104]:
data = pd.DataFrame(documents['train']).dropna(inplace=True)
data.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [102]:
#train_data = documents['train']['text'].apply(lambda x : str(x).lower())
data = pd.DataFrame(documents['train']).dropna(inplace=True)
data = data.reset_index(drop=True)
train_data = data['text'].str.lower()
train_labels = data['sentiment'].apply(lambda x : transform_target(x))

for i in range(3):
    print("{}\t{}".format(train_data[i], train_labels[i]))

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [55]:
test_data = documents['test']['text'].apply(lambda x : str(x).lower())
test_labels = documents['test']['sentiment'].apply(lambda x : transform_target(x))

for i in range(3):
    print("{}\t{}".format(test_data[i], test_labels[i]))

last session of the day  http://twitpic.com/67ezh	0
 shanghai is also really exciting (precisely -- skyscrapers galore). good tweeps in china:  (sh)  (bj).	1
recession hit veronique branquinho, she has to quit her company, such a shame!	-1


In [56]:

# Representation of the data using TF-IDF
vectorizer = TfidfVectorizer()
vectorised_train_data = vectorizer.fit_transform(train_data)
vectorised_test_data = vectorizer.transform(test_data)

# Train the classifier given the training data
classifier = LinearSVC()
classifier.fit(vectorised_train_data, train_labels)

# Predict the labels for the test documents (not used for training)
print(classifier.predict(vectorised_test_data))

[ 0  1 -1 ... -1  1  1]


In [77]:
len(vectorizer.vocabulary_)

26439

In [58]:
from sklearn.metrics import classification_report

In [59]:
cr = classification_report(test_labels, classifier.predict(vectorised_test_data))

In [61]:
print(cr)

              precision    recall  f1-score   support

          -1       0.67      0.67      0.67      1001
           0       0.64      0.66      0.65      1430
           1       0.76      0.73      0.75      1103

    accuracy                           0.68      3534
   macro avg       0.69      0.69      0.69      3534
weighted avg       0.69      0.68      0.68      3534



In [62]:
from sklearn.metrics import f1_score

In [63]:
f1_score(test_labels, classifier.predict(vectorised_test_data), average='macro')

0.6876580952163184

In [65]:
from sklearn.feature_extraction.text import CountVectorizer

In [66]:
count_vect = CountVectorizer()

In [67]:
X_train_counts = count_vect.fit_transform(train_data)

In [71]:
len(count_vect.vocabulary_)

26439

In [82]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf.fit(vectorised_train_data, train_labels)

RandomForestClassifier(n_estimators=50, n_jobs=-1)

In [84]:
f1_score(test_labels, clf.predict(vectorised_test_data), average='macro')

0.6830021272787273

In [85]:
clf.predict(vectorised_test_data)[0:5]

array([ 0,  1, -1,  1, -1])

In [83]:
from sklearn.metrics import roc_auc_score, roc_curve, auc
score = clf.predict_proba(vectorised_test_data)
fpr, tpr, th = roc_curve(test_labels, score[:, 1])

ValueError: multiclass format is not supported

In [73]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin

from xgboost import XGBClassifier

In [74]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [75]:
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [87]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')), #Sélection de la colonne à transformer (corpus)
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english'), #Sélection de la colonne à transformer (corpus)
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB : linear dimensionality reduction by means of truncated singular value decomposition (SVD)
        ])),
        ('words', Pipeline([
            ('wordext', NumberSelector('TotalWords')),
            ('wscaler', StandardScaler()),
        ])),
    ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('clf', RandomForestClassifier()),
    ])

In [97]:
X_train = pd.DataFrame(train_data, columns=['text'])
y_train=train_labels
X_test = pd.DataFrame(test_data, columns=['text'])
y_test=test_labels

In [98]:
X_train.head()

,text
0,"i`d have responded, if i were going"
1,sooo sad i will miss you here in san diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"sons of ****, why couldn`t they put them on t..."


In [99]:
classifier.fit(X_train, y_train)
preds = classifier.predict(X_test)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
text_clf = Pipeline([
     ('vect', TfidfVectorizer()),
     ('tfidf', TfidfTransformer()),
     ('clf', MultinomialNB()),
])